In [1]:
import requests
import json
import pandas as pd
import psycopg2 

In [2]:
city_names = ['Bonn', 'Düsseldorf', 'Nürnberg', 'Köln', 'Leipzig', 'Dresden', 'Karlsruhe', 'Berlin', 'Bremen', 'Bilbao', 'Ostrava', 'Duisburg', 'Dortmund', 'Mannheim', 'Heidelberg', 'Glasgow', 'Freiburg', 'Žilina', 'Ranchi', 'Las Palmas de Gran Canaria', 'Cardiff', 'Delhi', 'Marburg', 'Hannover', 'Luzern', 'Pittsburgh', 'Gießen', 'Hoboken', 'Kassel', 'Kiel']

In [97]:
def get_cities_popul(city_names):
    res_cities = []
    res_pops = []
    error_cities = []

    for name in city_names:
        api_url = 'https://countriesnow.space/api/v0.1/countries/population/cities'
        response = requests.post(api_url, data=[('city',  name)])
        if response.status_code == requests.codes.ok:
            resp = json.loads(response.text)
            res_cities.append(name)
            res_pops.append(round(float(resp['data']['populationCounts'][0]['value'])))
        else:
            error_cities.append(name)

    return pd.DataFrame({'city': res_cities, 'population': res_pops}), error_cities

In [98]:
# get cities populations
city_popul, error_cities = get_cities_popul(city_names)

In [48]:
print(f"No data found for: {error_cities}")

No data found for: ['Freiburg', 'Žilina', 'Marburg', 'Pittsburgh', 'Gießen', 'Hoboken']


In [49]:
# manual missing data inputation based on wikipedia.com
city_popul.loc[len(city_popul)] = ['Freiburg', 230940]
city_popul.loc[len(city_popul)] = ['Žilina', 81094]
city_popul.loc[len(city_popul)] = ['Marburg', 73836]
city_popul.loc[len(city_popul)] = ['Pittsburgh', 302971]
city_popul.loc[len(city_popul)] = ['Gießen', 83628]
city_popul.loc[len(city_popul)] = ['Hoboken',  58690]

In [50]:
city_popul

,city,population
0,Bonn,309869
1,Düsseldorf,593682
2,Nürnberg,495121
3,Köln,1024373
4,Leipzig,520838
5,Dresden,525105
6,Karlsruhe,296033
7,Berlin,3375222
8,Bremen,546451
9,Bilbao,350493


In [3]:
# connect to the database
try:
    conn = psycopg2.connect("dbname='bda-2022-nextbike-data' user='postgres' host='34.118.39.208' password='bda-2022-postgres'")
    print('Success connecting to the DB')
except:
    print('I am unable to connect to the database')

cursor = conn.cursor()

Success connecting to the DB


In [53]:
# create table
try:
    cursor.execute("""CREATE TABLE cities_populations (
                    city VARCHAR(200), 
                    population INTEGER, 
                    PRIMARY KEY(city))""")
    conn.commit()
except Exception as e:
    cursor.execute("ROLLBACK")
    print(e)

In [89]:
# insert cities populations values
cities_values = ""
for i in range(len(city_popul)):
    cities_values = cities_values + f"('{city_popul.iloc[i].city}', {city_popul.iloc[i].population}),"
cities_values = cities_values[:-1]

try:
    cursor.execute("INSERT INTO cities_populations VALUES " + cities_values)
    conn.commit()
except Exception as e:
    cursor.execute("ROLLBACK")
    print(e)


In [4]:
# test whether values are well inserted
cursor.execute("""SELECT * FROM cities_populations""")
rows = cursor.fetchall()
print(rows)

[('Bonn', 309869), ('Düsseldorf', 593682), ('Nürnberg', 495121), ('Köln', 1024373), ('Leipzig', 520838), ('Dresden', 525105), ('Karlsruhe', 296033), ('Berlin', 3375222), ('Bremen', 546451), ('Bilbao', 350493), ('Ostrava', 297421), ('Duisburg', 486816), ('Dortmund', 572087), ('Mannheim', 294627), ('Heidelberg', 150335), ('Glasgow', 577869), ('Ranchi', 847093), ('Las Palmas de Gran Canaria', 382578), ('Cardiff', 327706), ('Delhi', 9879172), ('Hannover', 514137), ('Luzern', 78786), ('Kassel', 192874), ('Kiel', 239866), ('Freiburg', 230940), ('Žilina', 81094), ('Marburg', 73836), ('Pittsburgh', 302971), ('Gießen', 83628), ('Hoboken', 58690)]


In [3]:
conn.close()

NameError: name 'conn' is not defined

In [9]:
# test whether values are well inserted
cursor.execute("""SELECT * FROM intervals_usage limit 10""")
rows = cursor.fetchall()
print(rows[0:1])

[(datetime.datetime(2019, 10, 10, 16, 2, 30), datetime.datetime(2019, 10, 10, 16, 7, 30), 5150273, 'Germany', 547, 'Bonn', 50.732875555556, 7.1172566666667, 0, 1)]


In [7]:
cursor.execute("ROLLBACK")